<h1>Audio</h1>

Wieso sind Töne und Geräusche wichtig?<br>
Auch diese können wie normale Daten wie Text oder Bilder mit einem neuralen Netz verarbeitet werden. Neurale Netze sind dafür sehr gut geeignet, diese können verschiedenste Features erfassen anders als die einfacheren Machine Learning Konzepte. Besonders wenn es komplexere Zusammenhänge gibt wie bei Bildern.

Es gibt unterschiedlichste Use-Cases wie:
- Spracherkennung (Wörter, Befehle, Audio-to-text)
- Klassifizierung von Tönen wie Musik.
- Umgebungswahrnehmung
- Geräusche differenzieren und Unterschiede erkennen

Um den Anfang zu machen, wollen wir als Ziel ein Model trainieren, dass Wörter erkennen soll.:
- Ein ASR Model (Automatic-speech-recognition)

Als Dataset nutzen wir:
> Speech Commands dataset (Warden, 2018) <br>
> https://www.tensorflow.org/datasets/catalog/speech_commands [Letzter Zugriff: 12.08.2024]

Dieses Dataset enthält folgende Wörter im Audioformat: "down", "go", "left", "no", "right", "stop", "up" und "yes"

In [1]:
# Imports 
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from IPython import display

In [2]:
tf.__version__

'2.17.0'

Dan laden wir das Dataset herunter.

In [5]:
DATASET_PATH = 'data/mini_speech_commands'

data_dir = pathlib.Path(DATASET_PATH)
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')

182082353/182082353 ━━━━━━━━━━━━━━━━━━━━ 16s 0us/step


In [8]:
# Dataset enthält Ordner mit Audiodateien.
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = commands[(commands != 'README.md') & (commands != '.DS_Store')]
print(f'Kommandos: {commands}')

Kommandos: ['down' 'go' 'left' 'no' 'right' 'stop' 'up' 'yes']
